<a href="https://colab.research.google.com/github/mrngochuu/Emotion-Recognition/blob/master/Emotion_tranfer_learning_VGG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Download dataset from your google drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cp /content/drive/My\ Drive/Emotion-face.zip /content

Prepare dataset

In [ ]:
import cv2
import os

# unzip file
!unzip ./Emotion-face.zip

# Crop face image dataset to gray image
folder_path = "./Emotion-face"
face_haar_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
for root, dirs, files in os.walk(folder_path, topdown=False):
    for file_name in files:
      if(file_name != '.DS_Store'):
        print('path: ' + str(root) + '/' +str(file_name))
        image = cv2.imread(os.path.join(root,file_name))
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        faces_detected = face_haar_cascade.detectMultiScale(gray,
        scaleFactor=1.3,
        minNeighbors=3,
        minSize=(30, 30))
        for (x,y,w,h) in faces_detected:
          cv2.rectangle(gray, (x, y), (x+w, y+h), 
                  (0, 255, 0), 2)
          faces = gray[y:y + h, x:x + w]
          cv2.imwrite(os.path.join(root,file_name),faces)
        print('path: ' + str(root) + '/' +str(file_name))

Streaming output truncated to the last 5000 lines.
  inflating: Emotion-face/train/angry/Training_46959039.jpg  
  inflating: __MACOSX/Emotion-face/train/angry/._Training_46959039.jpg  
  inflating: Emotion-face/train/angry/Training_3397618.jpg  
  inflating: __MACOSX/Emotion-face/train/angry/._Training_3397618.jpg  
  inflating: Emotion-face/train/angry/Training_19848415.jpg  
  inflating: __MACOSX/Emotion-face/train/angry/._Training_19848415.jpg  
  inflating: Emotion-face/train/angry/Training_5109662.jpg  
  inflating: __MACOSX/Emotion-face/train/angry/._Training_5109662.jpg  
  inflating: Emotion-face/train/angry/Training_24978929.jpg  
  inflating: __MACOSX/Emotion-face/train/angry/._Training_24978929.jpg  
  inflating: Emotion-face/train/angry/Training_35121977.jpg  
  inflating: __MACOSX/Emotion-face/train/angry/._Training_35121977.jpg  
  inflating: Emotion-face/train/angry/Training_19345091.jpg  
  inflating: __MACOSX/Emotion-face/train/angry/._Training_19345091.jpg  
  inflat

Import liblary

In [ ]:
import tensorflow as tf
import keras
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Flatten
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image

In [ ]:
Setup the data generators

In [ ]:
train_data_dir = "./Emotion-face/train"
datagen = ImageDataGenerator(
        zoom_range=0.1,
        rescale=1. / 255,
        rotation_range=10,
        shear_range=0.2,
        horizontal_flip=True,
        brightness_range=[0.2,2.0],
        fill_mode='nearest')
train_generator = datagen.flow_from_directory(train_data_dir,
                                              target_size=(224,224),
                                              batch_size=4,
                                              color_mode="rgb",
                                              class_mode='categorical',
                                              shuffle=True,
                                              seed=42)

Found 29045 images belonging to 7 classes.


In [ ]:
test_data_dir = "./Emotion-face/test"
datagen = ImageDataGenerator(
        zoom_range=0.1,
        rescale=1. / 255,
        rotation_range=10,
        shear_range=0.2,
        horizontal_flip=True,
        brightness_range=[0.2,2.0],
        fill_mode='nearest')
test_generator = datagen.flow_from_directory(test_data_dir,
                                             target_size=(224,224),
                                             batch_size=8,
                                             color_mode="rgb",
                                             class_mode='categorical',
                                             shuffle=True,
                                             seed=42)

Found 7275 images belonging to 7 classes.


Using VGGFace - with vgg16 model

In [ ]:
!pip install keras_vggface
!pip install keras_applications
from keras_vggface.vggface import VGGFace
from tensorflow.keras.applications.vgg16 import VGG16

vggmodel = VGGFace(model='vgg16', include_top=True)
vggmodel.summary()

Model: "vggface_vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 224, 224, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 112, 112, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 112, 112, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 56, 56, 128)     

Freeze the required layers

In [ ]:
for layers in (vggmodel.layers)[:19]:
    print(layers)
    layers.trainable = False

Create a new model

In [ ]:
X= vggmodel.layers[-2].output
# x = vggmodel.output
# x = GlobalAveragePooling2D () (x)
# x = Dense (100, activation='relu') (x)
# x = Dropout (0.15) (x)
# x = Dense (60, activation='relu') (x)
# x = Dense (30, activation='relu') (x)
predictions = Dense(7, activation="softmax")(X)
model_final = Model(inputs = vggmodel.input, outputs = predictions)

In [ ]:
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(learning_rate=0.0001, momentum=0.9), metrics=["accuracy"])
model_final.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 224, 224, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 112, 112, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 112, 112, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 56, 56, 128)       0   

Check GPU info

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Fri Jul  9 19:22:39 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P0    28W /  70W |   6960MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Traning model - using early stopping based on val_accuracy

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=7, verbose=1, mode='auto')
stats = model_final.fit_generator(generator= train_generator, epochs= 50, validation_data= test_generator, callbacks=[checkpoint,early])
# checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto')
# early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=7, verbose=1, mode='auto')
# stats = model_final.fit(train_generator, batch_size=30, epochs=30, verbose=1, validation_data)
model_final.save("emotion_prediction_vggface_5.h5")


Epoch 1/50


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


7262/7262 [==============================] - 586s 80ms/step - loss: 1.8676 - accuracy: 0.2489 - val_loss: 1.7645 - val_accuracy: 0.2471

Epoch 00001: val_accuracy improved from -inf to 0.24715, saving model to vgg16_1.h5
Epoch 2/50
7262/7262 [==============================] - 614s 85ms/step - loss: 1.6778 - accuracy: 0.3198 - val_loss: 1.4111 - val_accuracy: 0.4786

Epoch 00002: val_accuracy improved from 0.24715 to 0.47863, saving model to vgg16_1.h5
Epoch 3/50
7262/7262 [==============================] - 587s 81ms/step - loss: 1.3476 - accuracy: 0.5050 - val_loss: 1.2841 - val_accuracy: 0.5287

Epoch 00003: val_accuracy improved from 0.47863 to 0.52866, saving model to vgg16_1.h5
Epoch 4/50
7262/7262 [==============================] - 597s 82ms/step - loss: 1.2398 - accuracy: 0.5393 - val_loss: 1.2316 - val_accuracy: 0.5511

Epoch 00004: val_accuracy improved from 0.52866 to 0.55107, saving model to vgg16_1.h5
Epoch 5/50
7262/7262 [==============================] - 591s 81ms/step - l

Check Performance

In [ ]:
import matplotlib.pyplot as plt
plt.plot(stats.history["accuracy"])
plt.plot(stats.history['val_accuracy'])
plt.plot(stats.history['loss'])
plt.plot(stats.history['val_loss'])
plt.title("Model Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epochs")
plt.legend(["Accuracy","Validation Accuracy","Loss","Validation Loss"])
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 12
fig_size[1] = 12
plt.rcParams["figure.figsize"] = fig_size
plt.savefig('traningAcc.png')

plt.show()

NameError: ignored

Using weight file of above trained model to test image

In [ ]:
!pip install mtcnn

from keras.models import load_model

model = load_model("./emotion_prediction_vggface_5.h5")


In [ ]:
from matplotlib.patches import Rectangle
from mtcnn.mtcnn import MTCNN
from matplotlib import pyplot as plt
from PIL import Image
from numpy import asarray
import numpy as np
from skimage import color

def highlight_faces(image_path):
  image = plt.imread(image_path)
  detector = MTCNN()
  faces = detector.detect_faces(image)
  plt.imshow(image)
  ax = plt.gca()
  fig_size = plt.rcParams["figure.figsize"]
  fig_size[0] = 12
  fig_size[1] = 12
  plt.rcParams["figure.figsize"] = fig_size
  for face in faces: 
    if face['confidence'] > 0.95:
      x, y, width, height = face['box']
      cropped_face = extract_face(face,image)
      emotion_str = preprocess_otg(cropped_face)
      face_border = Rectangle((x, y), width, height,
                        fill=False, color=(1,0.859,0.604))
      plt.text(x , y , emotion_str,
                family="monospace",
                weight = "bold",
                color = "w", 
                bbox=dict(boxstyle="round",
                          fc=(1, 0.647, 0.004),
                          ec=(1, 0.647, 0.004)
                          ))
      ax.add_patch(face_border)
  plt.axis('off')
  plt.show()

def extract_face(face,image,required_size=(224,224)):

  x1, y1, width, height = face['box']
  x2, y2 = x1 + width, y1 + height
  face_boundary = image[y1:y2, x1:x2]
  face_image = Image.fromarray(face_boundary)
  face_image = face_image.resize(required_size)
  face_array = asarray(face_image)
  return face_array

def preprocess_otg(face_img):
  emotion_dict= {'Angry': 0, 'Sad': 5, 'Neutral': 4, 'Disgust': 1, 'Surprise': 6, 'Fear': 2, 'Happy': 3}
  image = rgb_gray(face_img)
  image = color.gray2rgb(image)
  image = image.reshape((1, 224, 224, 3))
  predicted_class = np.argmax(model.predict(image))
  label_map = dict((v,k) for k,v in emotion_dict.items()) 
  predicted_label = label_map[predicted_class]
  return predicted_label

def rgb_gray(face):
  img = color.rgb2gray(face)
  return img

In [ ]:
input_image = "./sample_images/emotions.jpg"
highlight_faces(input_image)